In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
PATH = 'synthetic_ride_hail_phnom_penh.csv'

data = pd.read_csv(PATH)

data.info()

### Data Preprocessing

### Label Encode for Ordinal

In [ ]:
# traffic_level has a natural order
traffic_order = {'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
data['traffic_level_encoded'] = data['traffic_level'].map(traffic_order)

### One-Hot Encode for Norminal


In [ ]:
# ── Nominal (One-Hot) ────────────────────────────
data = pd.get_dummies(data, columns=['vehicle_type', 'weather', 'time_of_day'],
                    drop_first=True)

# ── Date/Time Features ───────────────────────────
data['hour']        = pd.to_datetime(data['request_time']).dt.hour
data['date']        = pd.to_datetime(data['date'])
data['day_of_week'] = data['date'].dt.dayofweek   # 0–6 (label encode naturally)
data['is_weekday']  = (data['day_of_week'] < 5).astype(int)
# ❌ Skip is_weekend (redundant with is_weekday)

# ── Drop originals + non-numeric string columns ──
data.drop(columns=[
    'traffic_level',
    'date',
    'request_time',
    'arr_time',       # ← this is the culprit '12:15'
    'trip_id',        # ← ID, not a feature
    'dept_lat', 'dept_lon', 'arr_lat', 'arr_lon',  # ← coordinates (optional)
    'day',            # ← already have day_of_week
], inplace=True)

In [ ]:
# Convert any remaining bool columns to int (some sklearn versions don't like bool)
# Turn bool into int
bool_cols = data.select_dtypes(include='bool').columns
data[bool_cols] = data[bool_cols].astype(int)

# Final check — should print nothing if all clear
print("Remaining non-numeric columns:", 
      data.select_dtypes(exclude='number').columns.tolist())

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
target = 'fare_usd'     # we want to predict the fare_usd

X = data.drop(columns=[target])
y = data[target]

print("Shape of Features:", X.shape)
print("Shape of Target:", y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Shape of X Train:", X_train.shape)
print("Shape of X Test:", X_test.shape)
print("Shape of y Train", y_train.shape)
print("Shape of y Test", y_test.shape)

### Standard Scaled our Features

In [ ]:
# Run this BEFORE scaling to inspect remaining string columns
print(X_train.dtypes[X_train.dtypes == 'object'])
print(X_train.dtypes[X_train.dtypes == 'category'])  # time_of_day might still be category

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
models = {
    "Linear Regression":    LinearRegression(),
    "Decision Tree":        DecisionTreeRegressor(random_state=42),
    "Random Forest":        RandomForestRegressor(n_estimators=100, random_state=42),
}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)

In [ ]:
for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    print(f"--- {name} ---")
    print(f"  MAE  : {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"  RMSE : {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
    print(f"  R²   : {r2_score(y_test, y_pred):.4f}")

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.linear_model    import LinearRegression
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor
import xgboost                as xgb
import numpy                  as np

# ── Cross-Validation Setup ────────────────────────────────────────────────────
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ── Hyperparameter Grids ──────────────────────────────────────────────────────
param_grids = {
    "Linear Regression": {
        'model': LinearRegression(),
        'params': {
            'fit_intercept': [True, False],
            'positive':      [True, False],
        }
    },

    "Decision Tree": {
        'model': DecisionTreeRegressor(random_state=42),
        'params': {
            'max_depth':        [3, 5, 10, None],
            'min_samples_split':[2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features':     ['sqrt', 'log2', None],
        }
    },

    "Random Forest": {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators':     [50, 100, 200],
            'max_depth':        [3, 5, 10, None],
            'min_samples_split':[2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features':     ['sqrt', 'log2'],
        }
    },

    "XGBoost": {
        'model': xgb.XGBRegressor(random_state=42, verbosity=0),
        'params': {
            'n_estimators':     [50, 100, 200],
            'max_depth':        [3, 5, 7],
            'learning_rate':    [0.01, 0.05, 0.1, 0.2],
            'subsample':        [0.7, 0.8, 1.0],
            'colsample_bytree': [0.7, 0.8, 1.0],
            'reg_alpha':        [0, 0.1, 0.5],   # L1 regularization
            'reg_lambda':       [1, 1.5, 2.0],   # L2 regularization
        }
    },
}

# ── Run GridSearchCV for Each Model ──────────────────────────────────────────
best_models    = {}
cv_results_all = {}

for name, config in param_grids.items():
    print(f"\n🔍 Tuning: {name} ...")

    grid_search = GridSearchCV(
        estimator  = config['model'],
        param_grid = config['params'],
        cv         = kf,
        scoring    = 'neg_mean_absolute_error',  # MAE as scoring metric
        n_jobs     = -1,                          # use all CPU cores
        verbose    = 1,
    )

    grid_search.fit(X_train_scaled, y_train)

    best_models[name]    = grid_search.best_estimator_
    cv_results_all[name] = grid_search

    print(f"   ✅ Best Params : {grid_search.best_params_}")
    print(f"   ✅ Best CV MAE : ${-grid_search.best_score_:.4f}")

In [ ]:
# ── Evaluate All Tuned Models on Test Set ────────────────────────────────────
print("\n" + "=" * 70)
print("📊 TUNED MODEL PERFORMANCE ON TEST SET")
print("=" * 70)

results = []

for name, model in best_models.items():
    y_pred = model.predict(X_test_scaled)

    mae  = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2   = r2_score(y_test, y_pred)

    # Custom tolerance accuracy
    acc_050 = np.mean(np.abs(y_test.values - y_pred) <= 0.50) * 100
    acc_100 = np.mean(np.abs(y_test.values - y_pred) <= 1.00) * 100

    results.append({
        'Model': name,
        'MAE':   mae,
        'RMSE':  rmse,
        'R²':    r2,
        'Acc±$0.50': acc_050,
        'Acc±$1.00': acc_100,
    })

    print(f"\n--- {name} ---")
    print(f"   MAE        : ${mae:.4f}")
    print(f"   RMSE       : ${rmse:.4f}")
    print(f"   R²         : {r2:.4f}")
    print(f"   Acc ±$0.50 : {acc_050:.1f}%")
    print(f"   Acc ±$1.00 : {acc_100:.1f}%")

# ── Results Summary Table ─────────────────────────────────────────────────────
results_df = pd.DataFrame(results).set_index('Model')
print("\n")
print(results_df.round(4))

In [ ]:
# Actual vs Predicted plot
import matplotlib.pyplot as plt

best_model = models["Random Forest"]
y_pred = best_model.predict(X_test_scaled)

plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual fare_usd")
plt.ylabel("Predicted fare_usd")
plt.title("Actual vs Predicted Fare")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

importances = best_model.feature_importances_
feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)

sns.barplot(x=feat_imp.values[:10], y=feat_imp.index[:10])
plt.title("Top 10 Feature Importances")
plt.show()

In [ ]:
feat_imp

In [ ]:
# Make a prediction on test set using the best model
y_pred_test = best_model.predict(X_test_scaled)

# Convert scaled features back to DataFrame for display
X_test_display = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Create a detailed results DataFrame
detailed_results = X_test_display.copy()
detailed_results['Actual_Fare']    = y_test.values
detailed_results['Predicted_Fare'] = y_pred_test
detailed_results['Error']          = abs(y_test.values - y_pred_test)

print("=" * 100)
print("DETAILED FARE PREDICTIONS ON TEST SET")
print("=" * 100)
print("\nShowing key features that influence each prediction:\n")

# Display first 5 detailed predictions
for i in range(min(5, len(detailed_results))):
    row = detailed_results.iloc[i]

    print(f"\n{'─' * 100}")
    print(f"🛺 PREDICTION #{i+1}")
    print(f"{'─' * 100}")

    print(f"\n📍 Trip Details:")
    print(f"   Trip Distance:      {row['trip_distance_km']:.2f} (scaled)")
    print(f"   Actual Time:        {row['actual_time_min']:.2f} (scaled)")
    print(f"   Estimated Time:     {row['est_time_min']:.2f} (scaled)")
    print(f"   Wait Time:          {row['wait_time_min']:.2f} (scaled)")

    print(f"\n🌤️  Conditions:")
    print(f"   Traffic Level:      {row['traffic_level_encoded']:.2f} (encoded & scaled)")

    print(f"\n💰 Pricing:")
    print(f"   Surge Pricing:      {row['surge_pricing']:.2f} (scaled)")
    print(f"   Tip (USD):          {row['tip_usd']:.2f} (scaled)")

    print(f"\n⭐ Driver:")
    print(f"   Driver Rating:      {row['driver_rating']:.2f} (scaled)")

    print(f"\n📅 Timing:")
    print(f"   Hour:               {row['hour']:.2f}")
    print(f"   Day of Week:        {row['day_of_week']:.2f}")
    print(f"   Is Weekday:         {row['is_weekday']:.2f}")

    print(f"\n🎯 PREDICTION RESULTS:")
    print(f"   Actual Fare:        ${row['Actual_Fare']:.2f}")
    print(f"   Predicted Fare:     ${row['Predicted_Fare']:.2f}")
    print(f"   Error:              ${row['Error']:.2f}")

    # Accuracy indicator based on fare error
    if row['Error'] < 0.50:
        accuracy = "🟢 Excellent"
    elif row['Error'] < 1.00:
        accuracy = "🟡 Good"
    elif row['Error'] < 2.00:
        accuracy = "🟠 Fair"
    else:
        accuracy = "🔴 Poor"
    print(f"   Accuracy:           {accuracy}")

print(f"\n{'=' * 100}")

# ── Summary Statistics ────────────────────────────────────────────────────────
print("\n📊 OVERALL TEST SET PERFORMANCE")
print("=" * 100)
print(f"Total Samples:                {len(y_test)}")
print(f"Mean Actual Fare:             ${y_test.mean():.2f}")
print(f"Mean Predicted Fare:          ${y_pred_test.mean():.2f}")
print(f"\nMean Absolute Error:          ${mean_absolute_error(y_test, y_pred_test):.4f}")
print(f"Root Mean Squared Error:      ${np.sqrt(mean_squared_error(y_test, y_pred_test)):.4f}")
print(f"R² Score:                     {r2_score(y_test, y_pred_test):.4f}")

# ── Error Distribution ────────────────────────────────────────────────────────
errors = abs(y_test.values - y_pred_test)
print(f"\n📈 ERROR DISTRIBUTION:")
print(f"   Excellent (< $0.50):  {sum(errors < 0.50)} samples ({100*sum(errors < 0.50)/len(errors):.1f}%)")
print(f"   Good      ($0.50–$1): {sum((errors >= 0.50) & (errors < 1.00))} samples ({100*sum((errors >= 0.50) & (errors < 1.00))/len(errors):.1f}%)")
print(f"   Fair      ($1–$2):    {sum((errors >= 1.00) & (errors < 2.00))} samples ({100*sum((errors >= 1.00) & (errors < 2.00))/len(errors):.1f}%)")
print(f"   Poor      (> $2):     {sum(errors >= 2.00)} samples ({100*sum(errors >= 2.00)/len(errors):.1f}%)")